# Webscraping Notebook

## Installs

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
from bs4 import BeautifulSoup
import math
import numpy as np
import pandas as pd

## Scrape list of books and urls 

In [ ]:
page="1"
url="https://www.goodreads.com/shelf/show/fiction?page=" + page
url

In [ ]:
response=requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
book_titles = [book.text.strip() for book in soup.find_all('a', class_='bookTitle')]

In [ ]:
len(book_titles)

In [ ]:
book_titles[1]

In [ ]:
tkamb_title = book_titles[1]

In [ ]:
book_urls = ['https://www.goodreads.com' + book['href'] for book in soup.find_all('a', class_='bookTitle')]

In [ ]:
len(books)

In [ ]:
books[1]

## Get top reviews for each book

In [ ]:
response=requests.get(books[1])
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
type(soup)

In [ ]:
len(soup.find_all('section', 'ReviewText'))

In [ ]:
len(soup.find_all('article', 'ReviewCard'))

In [ ]:
review_containers = soup.find_all('article', 'ReviewCard')
reviews = []
for container in review_containers:
    review_text = container.find('section', 'ReviewText').text.strip()
    reviews.append(review_text)

In [ ]:
len(reviews)

In [ ]:
type(reviews[1])

In [ ]:
reviews[0]

In [ ]:
tkamb_reviews = reviews.copy()

## Get the average rating and title

In [ ]:
rating = soup.find('div', 'RatingStatistics__column')["aria-label"]

In [ ]:
rating

In [ ]:
float(rating[18:22])

In [ ]:
book_rating = float(rating[18:22])

In [ ]:
book_title = soup.select('h1', class_='Text Text__Title1')[0].text

## Saving title, rating, and 30 reviews together in a class

In [ ]:
class BookInfo:
    def __init__(self, rating, reviews, title):
        self.title = title
        self.rating = rating
        self.reviews = reviews
    
    def get_title(self):
        return self.title
    def get_rating(self):
        return self.rating
    def get_reviews(self):
        return self.reviews

In [ ]:
book1 = BookInfo(rating=tkamb_rating, reviews=tkamb_reviews, title=tkamb_title)

In [ ]:
book1.get_rating()

## Saving data into dataframe

In [ ]:
print(book_urls[0])

In [ ]:
response=requests.get(book_urls[0])
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
review_containers = soup.find_all('article', 'ReviewCard')
reviews = []
for container in review_containers:
    review_text = container.find('section', 'ReviewText').text.strip()
    reviews.append(review_text)

In [ ]:
print(reviews[0])

In [ ]:
rating = soup.find('div', 'RatingStatistics__column')["aria-label"][18:22]

In [ ]:
print(rating)

In [ ]:
book_title = soup.select('h1', class_='Text Text__Title1')[0].text

In [ ]:
print(book_title)

In [ ]:
book_dict = {
    'title': book_title,
    'rating': rating
}

In [ ]:
reviews_titles = [f'Review {str(num)}' for num in range(1,31)]

In [ ]:
reviews_titles

In [ ]:
for heading, review in zip(reviews_titles, reviews):
    book_dict[heading] = review

In [ ]:
print(book_dict['Review 5'])

In [ ]:
books_df = pd.DataFrame(book_dict, index=[0])

In [ ]:
books_df.info()

# Making functions

## Retrieving book urls

In [ ]:
def retrieve_books(num_books):
    page=1
    num_pages = math.floor(num_books/50)
    book_urls = []
    while page <= num_pages:
        url="https://www.goodreads.com/shelf/show/fiction?page=" + str(page)
        response=requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        for book in soup.find_all('a', class_='bookTitle'):
            book_urls.append('https://www.goodreads.com' + book['href'])
        page+=1
    if num_books%50:
        url="https://www.goodreads.com/shelf/show/fiction?page=" + str(page)
        response=requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        for book in soup.find_all('a', class_='bookTitle', limit=(num_books%50)):
            book_urls.append('https://www.goodreads.com' + book['href'])
    return book_urls

In [ ]:
len(book_urls)

## Retrieving top 30 reviews for each book and storing in dataframe

In [ ]:
def retrieve_data_for_book(book_url):
    book_dict = {}
    response=requests.get(book_url)
    soup = BeautifulSoup(response.content, "html.parser")
    review_containers = soup.find_all('article', 'ReviewCard')
    reviews = []
    for container in review_containers:
        review_text = container.find('section', 'ReviewText').text.strip()
        reviews.append(review_text)
    book_dict['rating'] = float(soup.find('div', class_='RatingStatistics__rating').text)
    book_dict['book_title'] = soup.select('h1', class_='Text Text__Title1')[0].text
    reviews_titles = [f'Review {str(num)}' for num in range(1,len(reviews)+1)]
    for heading, review in zip(reviews_titles, reviews):
        book_dict[heading] = review
    return book_dict

In [ ]:
def retrieve_data_for_all_books_in_df(book_urls):
    initial = True
    book_dicts=[]
    for book_url in book_urls:
        book_dicts.append(retrieve_data_for_book(book_url))
    books_df = pd.DataFrame.from_dict(book_dicts)
    return books_df

In [ ]:
books_df = retrieve_data_for_all_books_in_df(book_urls[:5])

## Testing Stuff

In [ ]:
book_urls[4]

In [ ]:
response=requests.get(book_urls[4])
soup = BeautifulSoup(response.content, "html.parser")
book_title = soup.select('h1', class_='Text Text__Title1')[0].text
book_title

In [ ]:
response=requests.get(book_urls[2])
book_rating = float(soup.find('div', class_='RatingStatistics__rating').text)
book_rating